对许多情况来说，单智能体可能已经足够，但更复杂的任务通常需要**协作和团队合作**，这也就是多智能体为什么必不可少的原因。

MetaGPT的**核心优势**也在于轻松灵活地开发一个智能体团队。


- 定义每个角色能够执行的预期动作。
- 基于标准作业程序（SOP）确保每个角色遵守它。通过使每个角色观察上游的相应输出结果，并为下游发布自己的输出结果。
- 初始化所有角色，创建一个带有环境的智能体团队，并使它们之间能够进行交互。


In [1]:
import re
from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.team import Team


def parse_code(rsp):
    pattern = r"```python(.*)```"
    match = re.search(pattern, rsp, re.DOTALL)
    code_text = match.group(1) if match else rsp
    return code_text


2024-11-19 08:46:07.493 | INFO     | metagpt.const:get_metagpt_package_root:21 - Package root set to /Users/Yam/Yam/llm_course/metagpt/MetaGPT


## 定义角色和动作


- SimpleCoder 具有 SimpleWriteCode 动作，接收用户的指令并编写主要代码
- SimpleTester 具有 SimpleWriteTest 动作，从 SimpleWriteCode 的输出中获取主代码并为其提供测试套件
- SimpleReviewer 具有 SimpleWriteReview 动作，审查来自 SimpleWriteTest 输出的测试用例，并检查其覆盖范围和质量

### Action

In [2]:
class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction}.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """
    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

In [3]:
class SimpleWriteTest(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Write {k} unit tests using pytest for the given function, assuming you have imported it.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteTest"

    async def run(self, context: str, k: int = 3):
        prompt = self.PROMPT_TEMPLATE.format(context=context, k=k)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

In [4]:
class SimpleWriteReview(Action):
    PROMPT_TEMPLATE: str = """
    Context: {context}
    Review the test cases and provide one critical comments:
    """

    name: str = "SimpleWriteReview"

    async def run(self, context: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context)

        rsp = await self._aask(prompt)

        return rsp

### Role

对于SimpleCoder

- 使用 set_actions 为Role配备适当的 Action。
- 多智能体操作逻辑：我们使Role _watch 来自用户或其他智能体的重要上游消息。

In [5]:
class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"
    goal: str = "write elegant and clean code"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._watch([UserRequirement])
        self.set_actions([SimpleWriteCode])

对于 SimpleTester

- 使用 set_actions 为SimpleTester配备 SimpleWriteTest 动作
- 使Role _watch 来自其他智能体的重要上游消息。
- 重写 _act 函数，我们希望SimpleTester将所有记忆用作编写测试用例的上下文，并希望有5个测试用例。

In [6]:
class SimpleTester(Role):
    name: str = "Bob"
    profile: str = "SimpleTester"
    goal: str = "test python code completely"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteTest])
        self._watch([SimpleWriteCode])
        # self._watch([SimpleWriteCode, SimpleWriteReview])  # feel free to try this too

    # 使用记忆
    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo

        # context = self.get_memories(k=1)[0].content # use the most recent memory as context
        context = self.get_memories()  # use all memories as context

        code_text = await todo.run(context, k=5)  # specify arguments
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

In [7]:
class SimpleReviewer(Role):
    name: str = "Charlie"
    profile: str = "SimpleReviewer"
    goal: str = "review python code as carefully as possible"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteReview])
        self._watch([SimpleWriteTest])

## 团队


将它们放在一起，设置一个 Team，并hire 它们。

In [8]:
import typer
from metagpt.logs import logger
from metagpt.team import Team
app = typer.Typer()

async def main(
    idea: str = typer.Argument(..., help="write a function that calculates the product of a list"),
    investment: float = typer.Option(default=3.0, help="Dollar amount to invest in the AI company."),
    n_round: int = typer.Option(default=5, help="Number of rounds for the simulation."),
):
    logger.info(idea)

    team = Team()
    team.hire(
        [
            SimpleCoder(),
            SimpleTester(),
            SimpleReviewer(),
        ]
    )

    team.invest(investment=investment)
    team.run_project(idea)
    await team.run(n_round=n_round)

In [31]:
await main("write a function that calculates the product of a list", 3.0, 3)

2024-11-19 08:41:21.850 | INFO     | __main__:main:11 - write a function that calculates the product of a list
2024-11-19 08:41:21.967 | INFO     | metagpt.team:invest:93 - Investment: $3.0.
2024-11-19 08:41:21.967 | INFO     | metagpt.roles.role:react:522 - Role.react mode: RoleReactMode.REACT
2024-11-19 08:41:21.968 | INFO     | metagpt.roles.role:_react:471 - Role._react todo: True
2024-11-19 08:41:21.968 | INFO     | metagpt.roles.role:_act:403 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)
2024-11-19 08:41:21.969 | INFO     | metagpt.actions.action:_aask:109 - action.Action._aaask: system message: None
2024-11-19 08:41:21.969 | INFO     | metagpt.provider.base_llm:aask:151 - base_llm.BaseLLM.aask  message: [{'role': 'system', 'content': 'You are a SimpleCoder, named Alice, your goal is write elegant and clean code. '}, {'role': 'user', 'content': "\n    Write a python function that can [Message(id='623c512bd9204e7e8e46beefbae9834d', content='write a function that cal

OpenAILLM Start Streaming Chunks:

```python
def product_of_list(lst):
    result = 1
    for num in

2024-11-19 08:41:23.504 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:41:23.506 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x2904e0110>
2024-11-19 08:41:23.508 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: ```python
def product_of_list(lst):
    result = 1
    for num in lst:
        result *= num
    return result
```
2024-11-19 08:41:23.510 | INFO     | metagpt.roles.role:run:558 - role.run rsp: id='5be0723e1f0e40bfb21cf7b37bfda19b' content='\ndef product_of_list(lst):\n    result = 1\n    for num in lst:\n        result *= num\n    return result\n' instruct_content=None role='SimpleCoder' cause_by='__main__.SimpleWriteCode' sent_from='__main__.SimpleCoder' send_to={'<all>'}
2024-11-19 08:41:23.512 | INFO     | metagpt.roles.role:run:554 - Alice(SimpleCoder): no news. waiting.
2024-11-19 08:41:23.5

 lst:
        result *= num
    return result
```OpenAILLM End Streaming Chunks:
OpenAILLM Start Streaming Chunks:

```python
import pytest

def test_product_of_list_with_positive_numbers():
    assert product_of_list([1, 2, 3, 4]) == 24

def test_product_of_list_with_negative_numbers():
    assert product_of_list([-1, -2, -3, -4]) == 24

def test_product_of_list_with_zero():
    assert product_of_list([1, 2, 0, 3]) == 0

def test_product_of_list_with_float_numbers():
    assert product_of_list([1.5, 2.0, 3.0]) == 9.0

def test_product_of_list_with_empty_list():
    assert product_of_list([]) == 1
```

2024-11-19 08:41:39.101 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:41:39.103 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x28741acd0>
2024-11-19 08:41:39.105 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: ```python
import pytest

def test_product_of_list_with_positive_numbers():
    assert product_of_list([1, 2, 3, 4]) == 24

def test_product_of_list_with_negative_numbers():
    assert product_of_list([-1, -2, -3, -4]) == 24

def test_product_of_list_with_zero():
    assert product_of_list([1, 2, 0, 3]) == 0

def test_product_of_list_with_float_numbers():
    assert product_of_list([1.5, 2.0, 3.0]) == 9.0

def test_product_of_list_with_empty_list():
    assert product_of_list([]) == 1
```
2024-11-19 08:41:39.106 | INFO     | metagpt.roles.role:run:558 - role.run rsp: id='5b31ae846c8b431a99803a9d1fb3

OpenAILLM End Streaming Chunks:
OpenAILLM Start Streaming Chunks:

The test cases provided cover a variety of scenarios, including positive numbers, negative numbers, zero, float numbers, and an empty list. However, there is a critical comment to be made regarding the test case `test_product_of_list_with_empty_list`:

The test case `test_product_of_list_with_empty_list` asserts that the product of an empty list should be 1. While this is mathematically correct, it might not be the desired behavior for all use cases. In some contexts, it could be more appropriate to return `None` or raise an exception when the input list is empty, as it indicates that there are no elements to multiply.

To improve the test case, consider adding an additional test to check for the desired behavior when the input list is empty. For example:

```python
def test_product_of_list_with_empty_list():
    with pytest.raises(ValueError):
        product_of_list([])
```

This test case would require modifying the 

2024-11-19 08:42:01.128 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:42:01.130 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x290d6d310>
2024-11-19 08:42:01.132 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: The test cases provided cover a variety of scenarios, including positive numbers, negative numbers, zero, float numbers, and an empty list. However, there is a critical comment to be made regarding the test case `test_product_of_list_with_empty_list`:

The test case `test_product_of_list_with_empty_list` asserts that the product of an empty list should be 1. While this is mathematically correct, it might not be the desired behavior for all use cases. In some contexts, it could be more appropriate to return `None` or raise an exception when the input list is empty, as it indicates that there are no 

 idea that multiplying an empty list of numbers is undefined.OpenAILLM End Streaming Chunks:


In [9]:
await main("编写一个计算 List 乘积的函数", 3.0, 3)

2024-11-19 08:46:32.846 | INFO     | __main__:main:11 - 编写一个计算 List 乘积的函数
2024-11-19 08:46:32.938 | INFO     | metagpt.team:invest:93 - Investment: $3.0.
2024-11-19 08:46:32.939 | INFO     | metagpt.roles.role:react:522 - Role.react mode: RoleReactMode.REACT
2024-11-19 08:46:32.939 | INFO     | metagpt.roles.role:_react:471 - Role._react todo: True
2024-11-19 08:46:32.940 | INFO     | metagpt.roles.role:_act:403 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)
2024-11-19 08:46:32.940 | INFO     | metagpt.actions.action:_aask:109 - action.Action._aaask: system message: None
2024-11-19 08:46:32.940 | INFO     | metagpt.provider.base_llm:aask:151 - base_llm.BaseLLM.aask  message: [{'role': 'system', 'content': 'You are a SimpleCoder, named Alice, your goal is write elegant and clean code. '}, {'role': 'user', 'content': "\n    Write a python function that can [Message(id='a8bce4ad41064f2d9381d0affe277131', content='编写一个计算 List 乘积的函数', instruct_content=None, role='Human', cause

OpenAILLM Start Streaming Chunks:

```python
def list_product(lst):
    result = 1
    for num in lst

2024-11-19 08:46:35.148 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:46:35.150 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x176292d50>
2024-11-19 08:46:35.151 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: ```python
def list_product(lst):
    result = 1
    for num in lst:
        result *= num
    return result
```
2024-11-19 08:46:35.154 | INFO     | metagpt.roles.role:run:558 - role.run rsp: id='ae29541a4edb421fa78be32275a09f7a' content='\ndef list_product(lst):\n    result = 1\n    for num in lst:\n        result *= num\n    return result\n' instruct_content=None role='SimpleCoder' cause_by='__main__.SimpleWriteCode' sent_from='__main__.SimpleCoder' send_to={'<all>'}
2024-11-19 08:46:35.155 | INFO     | metagpt.team:run:136 - max n_round=2 left.
2024-11-19 08:46:35.156 | INFO     | metagpt.roles.

:
        result *= num
    return result
```OpenAILLM End Streaming Chunks:
OpenAILLM Start Streaming Chunks:

```python
import pytest

def test_list_product_empty_list():
    assert list_product([]) == 1

def test_list_product_single_element():
    assert list_product([5]) == 5

def test_list_product_multiple_elements():
    assert list_product([1, 2, 3, 4]) == 24

def test_list_product_with_zero():
    assert list_product([1, 2, 0, 3]) == 0

def test_list_product_with_negative_numbers():
    assert list_product([-1, 2, -3,

2024-11-19 08:46:41.834 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:46:41.836 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x16bb78390>
2024-11-19 08:46:41.837 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: ```python
import pytest

def test_list_product_empty_list():
    assert list_product([]) == 1

def test_list_product_single_element():
    assert list_product([5]) == 5

def test_list_product_multiple_elements():
    assert list_product([1, 2, 3, 4]) == 24

def test_list_product_with_zero():
    assert list_product([1, 2, 0, 3]) == 0

def test_list_product_with_negative_numbers():
    assert list_product([-1, 2, -3, 4]) == 24
```
2024-11-19 08:46:41.838 | INFO     | metagpt.roles.role:run:558 - role.run rsp: id='4101525e77ec4ce9976c42b3c30592e9' content='\nimport pytest\n\ndef test_list_product_emp

 4]) == 24
```OpenAILLM End Streaming Chunks:
OpenAILLM Start Streaming Chunks:

The provided test cases cover a variety of scenarios, including an empty list, a list with a single element, a list with multiple elements, a list containing a zero, and a list with negative numbers. However, there is a critical comment to be made regarding the test cases:

**Critical Comment:** The test cases do not cover scenarios where the input list contains non-numeric types. The function `list_product` is designed to calculate the product of a list of numbers, but it does not have any error handling for non-numeric inputs. It would be beneficial to add test cases that check the function's behavior when the input list contains elements that are not numbers, such as strings or None. This would ensure that the function is robust

2024-11-19 08:46:50.472 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model moonshot-v1-auto not found in TOKEN_COSTS.
2024-11-19 08:46:50.475 | INFO     | metagpt.actions.action:_aask:111 - action.Action._aaask: self.llm: <metagpt.provider.openai_api.OpenAILLM object at 0x16bd46710>
2024-11-19 08:46:50.476 | INFO     | metagpt.actions.action:_aask:112 - action.Action._aaask: res: The provided test cases cover a variety of scenarios, including an empty list, a list with a single element, a list with multiple elements, a list containing a zero, and a list with negative numbers. However, there is a critical comment to be made regarding the test cases:

**Critical Comment:** The test cases do not cover scenarios where the input list contains non-numeric types. The function `list_product` is designed to calculate the product of a list of numbers, but it does not have any error handling for non-numeric inputs. It would be beneficial to add test cases that check the function's beh

 and can handle unexpected inputs gracefully.OpenAILLM End Streaming Chunks:


## 机制

![](multi_agents_flowchart.png)

- Role从Environment中`_observe` Message。
- 如果有一个Role `_watch` 的特定 Action 引起的 Message，那么这是一个有效的观察，触发Role的后续思考和操作。
- 在 `_think` 中，Role将选择其能力范围内的一个 Action 并将其设置为要做的事情。
- 在 `_act` 中，Role执行要做的事情，即运行 Action 并获取输出。
- 将输出封装在 Message 中，最终 `publish_message` 到 Environment，完成了一个完整的智能体运行。

在每个步骤中，无论是 _observe、_think 还是 _act，Role都将与其 Memory 交互，通过添加或检索来实现。

左边虚线框表明如果我们使 SimpleTester 同时 _watch SimpleWriteCode 和 SimpleWriteReview，则可以扩展 SOP。

## Homework

请自己执行一个一句话需求（建议游戏），并观察记录过程。